## Story Cloze: Bert for multiple choice

#### install tools

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

import datasets
from datasets import load_dataset, load_metric
from datasets import Dataset


In [ ]:
# # connect to huggingface to save model
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load training, validation, and test datasets
val_df = pd.read_csv('/content/drive/MyDrive/Project/cloze_test_val__winter2018-cloze_test_ALL_val - 1 - 1.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Project/cloze_test_test__winter2018-cloze_test_ALL_test - 1.csv')
train_df = pd.read_csv('/content/drive/MyDrive/Project/ROCStories_winter2017 - ROCStories_winter2017 (1).csv')


### Modify Training Dataset
1. create random incorrect ending
2. shuffle position of correct and incorrect ending
3. define lables based on shuffled positions

In [ ]:
# function to randomly shuffle each row across the correct and incorrect ending columns
def shuffle_row(row):
    values = row.values
    np.random.shuffle(values)
    return pd.Series(values, index=row.index)

In [ ]:
train_df['WrongSentence'] = train_df['sentence5'].sample(frac=1,replace=False).values + 'INCORRECT_ENDING'
train_df.iloc[:,-2:] = train_df.iloc[:,-2:].apply(shuffle_row,axis=1)

train_df['label'] = train_df.apply(lambda row: 1 if 'INCORRECT_ENDING' in row['sentence5'] else (0 if 'INCORRECT_ENDING' in row['WrongSentence'] else None), axis=1)
train_df.iloc[:,-3:-1] = train_df.iloc[:,-3:-1].applymap(lambda row: row.replace('INCORRECT_ENDING', ''))

train_df = train_df.rename(columns={'sentence5':'ending0','WrongSentence':'ending1'})

In [ ]:
# slim dataset to 20000 examples
train_df = train_df.iloc[:20000,:]

In [ ]:
### turn all train, validation, and test datasets into a dataset dictionary ###

train = pd.DataFrame({
    "InputStoryId" : train_df['storyid'].values,
    "sent1" : train_df.iloc[:,2:6].apply(lambda row: ' '.join(row), axis=1).values,
    "ending0" : train_df['ending0'].values,
    "ending1" : train_df['ending1'].values,
    "label" : train_df['label'].values
})


validation = pd.DataFrame({
    "InputStoryId" : val_df['InputStoryid'].values,
    "sent1" : val_df.iloc[:,1:5].apply(lambda row: ' '.join(row), axis=1).values,
    "ending0" : val_df.iloc[:,-3].values,
    "ending1" : val_df.iloc[:,-2].values,
    "label" : val_df.iloc[:,-1].values -1,
})

test = pd.DataFrame({
    "InputStoryId" : test_df['InputStoryid'].values,
    "sent1" : test_df.iloc[:,1:5].apply(lambda row: ' '.join(row), axis=1).values,
    "ending0" : test_df.iloc[:,-2].values,
    "ending1" : test_df.iloc[:,-1].values,
    "label" : [-1] * len(test_df),
})

train_dataset = Dataset.from_dict(train)
val_dataset = Dataset.from_dict(validation)
test_dataset = Dataset.from_dict(test)
my_dataset_dict = datasets.DatasetDict({"train":train_dataset,
                                        "validation" : val_dataset,
                                        "test":test_dataset})

## Look at some examples

In [ ]:
def show_one(example):
    print(f"Context: {example['sent1']}")
    print(f"  A - {example['ending0']}")
    print(f"  B - {example['ending1']}")
    print(f"\nGround truth: option {['A', 'B'][example['label']]}")

In [ ]:
show_one(my_dataset_dict['train'][0])

Context: David noticed he had put on a lot of weight recently. He examined his habits to try and figure out the reason. He realized he'd been eating too much fast food lately. He stopped going to burger places and started a vegetarian diet.
  A - She fled at night, with the help of the King's cupbearer.
  B - After a few weeks, he started to feel much better.

Ground truth: option B


## Preprocess

In [ ]:
### load tokenizer ###
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

#### function to create complete sentences with right and wrong ending and tokenize each

In [ ]:
def preprocess_function(examples):
    # copies the input sentence once for each ending - list of lists with 2 input sentences
    first_sentences = [[x] * 2 for x in examples['sent1']]

    # joins the endings as a list of lists
    ending_sentences = [list(x) for x in zip(examples['ending0'],examples['ending1'])]

    # flattens the lists
    first_sentences = sum(first_sentences, [])
    ending_sentences = sum(ending_sentences, [])

    # tokenizes each sentence and ending pair together
    tokenized_examples = tokenizer(first_sentences, ending_sentences, truncation=True)
    return {k: [v[i : i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

In [ ]:
tokenized_mydataset = my_dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1571 [00:00<?, ? examples/s]

Map:   0%|          | 0/1571 [00:00<?, ? examples/s]

#### Look at some tokenization and preprocessing outputs

In [ ]:
examples = my_dataset_dict["train"][:3]
features = preprocess_function(examples)
print(
    len(features["input_ids"]),
    len(features["input_ids"][0]),
    [len(x) for x in features["input_ids"][0]],
)

3 2 [70, 64]


In [ ]:
idx = 1
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(2)]

["[CLS] tom had a very short temper. one day a guest made him very angry. he punched a hole in the wall of his house. tom's guest became afraid and left quickly. [SEP] tom sat on his couch filled with regret about his actions. [SEP]",
 "[CLS] tom had a very short temper. one day a guest made him very angry. he punched a hole in the wall of his house. tom's guest became afraid and left quickly. [SEP] i ended up buying both containers. [SEP]"]

In [ ]:
show_one(my_dataset_dict['train'][idx])

Context: Tom had a very short temper. One day a guest made him very angry. He punched a hole in the wall of his house. Tom's guest became afraid and left quickly.
  A - Tom sat on his couch filled with regret about his actions.
  B - I ended up buying both containers.

Ground truth: option A


### Function to dynamically pad inputs

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="tf",
        )

        batch = {k: tf.reshape(v, (batch_size, num_choices, -1)) for k, v in batch.items()}
        batch["labels"] = tf.convert_to_tensor(labels, dtype=tf.int64)
        return batch

In [ ]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [
    {k: v for k, v in tokenized_mydataset["train"][i].items() if k in accepted_keys}
    for i in range(10)
]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

In [ ]:
tokenized_mydataset["train"].features["attention_mask"].feature.feature

Value(dtype='int8', id=None)

In [ ]:
[tokenizer.decode(batch["input_ids"][9][i].numpy().tolist()) for i in range(2)]

['[CLS] laura loved corn. so she decided to grow some in her backyard. the whole process of growing them made her very excited. but she realized that they required too much water. [SEP] my instincts were right when the teacher was arrested that year. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] laura loved corn. so she decided to grow some in her backyard. the whole process of growing them made her very excited. but she realized that they required too much water. [SEP] so laura quickly abandoned her corn garden idea. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

In [ ]:
show_one(my_dataset_dict["train"][9])

Context: Laura loved corn. So she decided to grow some in her backyard. The whole process of growing them made her very excited. But she realized that they required too much water.
  A - My instincts were right when the teacher was arrested that year.
  B - So Laura quickly abandoned her corn garden idea.

Ground truth: option B


#### Evaluation Metrics

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import create_optimizer

batch_size = 16
num_train_epochs = 2
total_train_steps = (len(tokenized_mydataset["train"]) // batch_size) * num_train_epochs
optimizer, schedule = create_optimizer(init_lr=5e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForMultipleChoice

model = TFAutoModelForMultipleChoice.from_pretrained("google-bert/bert-base-uncased")

All PyTorch model weights were used when initializing TFBertForMultipleChoice.

Some weights or buffers of the TF 2.0 model TFBertForMultipleChoice were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.weights[0].

TensorShape([768])

In [ ]:
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
tf_train_set = model.prepare_tf_dataset(
    tokenized_mydataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_mydataset["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
model.compile(
    optimizer=optimizer,
    metrics=["accuracy"]
    )  # No loss argument!

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
# from transformers.keras_callbacks import PushToHubCallback

# push_to_hub_callback = PushToHubCallback(
#     output_dir="my_awesome_model",
#     tokenizer=tokenizer,
# )


model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=2)

Epoch 1/2


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1250/1250 [==============================] - 705s 488ms/step - loss: 0.0320 - accuracy: 0.9904 - val_loss: 1.0536 - val_accuracy: 0.5939
Epoch 2/2
1250/1250 [==============================] - 603s 482ms/step - loss: 0.0032 - accuracy: 0.9991 - val_loss: 0.9247 - val_accuracy: 0.6022


In [ ]:
input_start = "Dan's parents were overweight. Dan was overweight as well. The doctors told his parents it was unhealthy. His parents understood and decided to make a change."
endings = [
    "They told a teacher and went to jail.",
    'They got themselves and Dan on a diet.'



]
full_sentences = [f"{input_start} {ending}" for ending in endings]

In [ ]:
tokenized = tokenizer(full_sentences, padding="longest", return_tensors="np")
tokenized = {key: np.expand_dims(array, 0) for key, array in tokenized.items()}

In [ ]:
outputs = model(tokenized).logits
answer = np.argmax(outputs)

print(f"The answer is choice {answer}: {endings[answer]}")

The answer is choice 0: They told a teacher and went to jail.


In [ ]:
outputs

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 1.2105044 , -0.48823348]], dtype=float32)>

In [ ]:
roc_stories_2016 = pd.read_csv('/content/drive/MyDrive/Project/ROCStories__spring2016 - ROCStories_spring2016.csv')

In [ ]:
roc_stories_2016.iloc[:,2:7].apply(lambda row: ' '.join(row), axis=1).values[0]

"Dan's parents were overweight. Dan was overweight as well. The doctors told his parents it was unhealthy. His parents understood and decided to make a change. They got themselves and Dan on a diet."